# Cross-Lingual Self-RAG

## 1. Installation and Imports

In [1]:
# Install required packages
!pip install -q transformers torch wikipedia-api googletrans==4.0.0-rc1 sentence-transformers
!pip install -q numpy pandas tqdm colorama huggingface-hub accelerate bitsandbytes

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-sdk 0.2.15 requires httpx>=0.25.2, but you have httpx 0.13.3 which is incompatible.
openai 2.9.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
langsmith 0.4

In [2]:
import os
import json
import time
import warnings
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from enum import Enum
import re

import numpy as np
import pandas as pd
from tqdm import tqdm
from colorama import Fore, Style, init

# Translation and NLP
from googletrans import Translator
import wikipediaapi
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
import torch
from huggingface_hub import login

# Initialize colorama for colored output
init(autoreset=True)
warnings.filterwarnings('ignore')

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\nAll packages imported successfully!")

Using device: cuda
GPU: Tesla T4
Memory: 15.83 GB

All packages imported successfully!


## 2. Hugging Face Authentication

In [3]:
HF_TOKEN = "REMOVED_TOKEN"
# Login to Hugging Face
try:
    login(token=HF_TOKEN)
    print("✅ Successfully logged in to Hugging Face!")
except Exception as e:
    print(f"⚠️ Warning: Could not login to Hugging Face: {e}")
    print("You may need to authenticate manually or some models might not be accessible.")

✅ Successfully logged in to Hugging Face!


## 3. Configuration and Data Classes

In [4]:
# Configuration
@dataclass
class Config:
    """Configuration for the Cross-Lingual Self-RAG system with Gemma"""
    # Language settings
    supported_languages: List[str] = None
    language_codes: Dict[str, str] = None

    # Retrieval settings
    wikipedia_top_k: int = 3
    max_passage_length: int = 500

    # NLI Model settings
    nli_model: str = "microsoft/deberta-v3-base"
    translation_batch_size: int = 8

    # Gemma model settings
    llm_model: str = "google/gemma-2b-it"  # Gemma 2B Instruction-tuned
    llm_temperature: float = 0.1
    llm_max_new_tokens: int = 512  # Maximum new tokens to generate
    llm_top_p: float = 0.95
    llm_top_k: int = 50
    llm_repetition_penalty: float = 1.1

    # Memory optimization settings
    load_in_8bit: bool = False  # Set to True if you have memory constraints
    load_in_4bit: bool = False  # Even more memory efficient
    device_map: str = "auto"  # Automatically map model to available devices
    torch_dtype: str = "float16"  # Use float16 for memory efficiency

    # Output settings
    verbose: bool = True
    save_intermediate: bool = True

    def __post_init__(self):
        self.supported_languages = ['tamil', 'hindi', 'kannada', 'marathi', 'english']
        self.language_codes = {
            'tamil': 'ta',
            'hindi': 'hi',
            'kannada': 'kn',
            'marathi': 'mr',
            'english': 'en'
        }

# Data classes for structured outputs
class SupportJudgment(Enum):
    SUPPORTED = "supported"
    PARTIALLY_SUPPORTED = "partially_supported"
    NOT_SUPPORTED = "not_supported"

@dataclass
class CritiqueResult:
    judgment: SupportJudgment
    rationale: str
    revised_answer: str
    confidence: float

@dataclass
class SelfRAGOutput:
    """Final output structure as specified in Step 9"""
    final_answer_indic: str  # Final answer in Indic language
    judgment_label: str  # supported / partial / not supported
    support_confidence: float  # Confidence score [0, 1]
    english_rationale: str  # Rationale for auditability

    # Additional fields for debugging
    initial_answer: str = None
    retrieved_context: str = None
    regenerated_answer: str = None
    english_answer: str = None
    nli_score: float = None

## 4. Wikipedia Retrieval Module

In [5]:
import requests
import html

class WikipediaRetriever:
    """On-the-fly Wikipedia retrieval via MediaWiki API (multilingual)"""

    def __init__(self, config):
        self.config = config

    # --- Keep / reuse your old language helpers if you already have them ---
    def resolve_language_key(self, language: str) -> str:
        # If you already have this elsewhere, reuse it.
        return (language or "").strip().lower()

    def normalize_token(self, token: str, language: str) -> str:
        token = re.sub(r"[^\w\u0B80-\u0BFF\u0900-\u097F\u0C80-\u0CFF]", "", token)

        if language == "tamil":
            token = re.sub(r"(இன்|யின்|வின்)$", "", token)
        elif language == "hindi":
            token = re.sub(r"(की|का|के)$", "", token)
        elif language in ("marathi", "kannada"):
            token = re.sub(r"(ची|चा|च्या|ರ|ಗೆ|ನಲ್ಲಿ)$", "", token)

        return token

    def extract_keywords(self, question: str, language: str) -> List[str]:
        stop_patterns = {
            'english': ['what', 'when', 'where', 'who', 'why', 'how', 'is', 'are', 'was', 'were'],
            'hindi': ['क्या', 'कब', 'कहाँ', 'कौन', 'क्यों', 'कैसे', 'है', 'हैं', 'था', 'थे'],
            'tamil': ['என்ன', 'எப்போது', 'எங்கே', 'யார்', 'ஏன்', 'எப்படி', 'எது'],
            'kannada': ['ಏನು', 'ಯಾವಾಗ', 'ಎಲ್ಲಿ', 'ಯಾರು', 'ಏಕೆ', 'ಹೇಗೆ'],
            'marathi': ['काय', 'केव्हा', 'कुठे', 'कोण', 'का', 'कसे']
        }

        raw_words = question.split()
        normalized_words = [self.normalize_token(w, language) for w in raw_words]

        stop_words = stop_patterns.get(language, [])
        stop_lower = {s.lower() for s in stop_words}

        keywords = [w for w in normalized_words if w and w.lower() not in stop_lower]
        return keywords[:5]

    # --- Your new MediaWiki API bits ---
    def mediawiki_search(self, language: str, query: str, limit: int) -> List[Dict]:
        code = self.config.language_codes.get(language)
        if not code:
            return []

        url = f"https://{code}.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "list": "search",
            "srsearch": query,
            "srlimit": limit,
            "format": "json",
            "utf8": 1,
        }

        r = requests.get(
            url, params=params, timeout=8,
            headers={"User-Agent": "CrossLingualRAG/1.0"}
        )
        r.raise_for_status()
        data = r.json()
        return data.get("query", {}).get("search", [])

    def fetch_extracts(self, language: str, pageids: List[int], chars: int) -> Dict[int, str]:
        code = self.config.language_codes.get(language)
        if not code or not pageids:
            return {}

        url = f"https://{code}.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "prop": "extracts|pageprops",
            "exintro": 1,
            "explaintext": 1,
            "exchars": chars,
            "pageids": "|".join(str(pid) for pid in pageids),
            "format": "json",
            "utf8": 1,
        }

        r = requests.get(
            url, params=params, timeout=8,
            headers={"User-Agent": "CrossLingualRAG/1.0"}
        )
        r.raise_for_status()

        pages = r.json().get("query", {}).get("pages", {})
        out: Dict[int, str] = {}

        for pid_str, pdata in pages.items():
            # filter disambiguation pages when possible
            if isinstance(pdata, dict) and pdata.get("pageprops", {}).get("disambiguation") is not None:
                continue
            try:
                out[int(pid_str)] = pdata.get("extract", "") or ""
            except Exception:
                pass
        return out

    def build_query_variants(self, question: str, language: str) -> List[str]:
        q_full = re.sub(
            r"\s+", " ",
            re.sub(r"[^\w\s\u0B80-\u0BFF\u0900-\u097F\u0C80-\u0CFF]", " ", question)
        ).strip()

        keywords = self.extract_keywords(question, language)

        variants: List[str] = []
        if q_full:
            variants.append(q_full)
        if keywords:
            variants.append(" ".join(keywords))

        for i in range(len(keywords) - 1):
            variants.append(keywords[i] + " " + keywords[i + 1])

        variants.extend(keywords)

        seen = set()
        uniq = []
        for v in variants:
            k = v.strip().lower()
            if v.strip() and k not in seen:
                seen.add(k)
                uniq.append(v.strip())

        return uniq[:10]

    def search_wikipedia(self, question: str, language: str, top_k: Optional[int] = None) -> List[str]:
        if top_k is None:
            top_k = self.config.wikipedia_top_k

        top_k = max(1, int(top_k))

        language = self.resolve_language_key(language)
        if language not in self.config.language_codes:
            if getattr(self.config, "verbose", False):
                print(f"[WikipediaRetriever] Unknown language key: {language}. "
                      f"Available: {list(self.config.language_codes.keys())}")
            return []

        variants = self.build_query_variants(question, language)
        if getattr(self.config, "verbose", False):
            print(f"[WikipediaRetriever] Query variants: {variants}")

        candidates = {}  # pageid -> (title, snippet)
        try:
            for q in variants:
                hits = self.mediawiki_search(language, q, limit=max(10, top_k * 3))
                for h in hits:
                    pid = int(h.get("pageid", 0))
                    if not pid or pid in candidates:
                        continue
                    title = h.get("title", "") or ""
                    snippet = h.get("snippet", "") or ""
                    snippet = html.unescape(re.sub(r"<[^>]+>", "", snippet))
                    candidates[pid] = (title, snippet)

                if len(candidates) >= top_k * 10:
                    break
        except Exception as e:
            if getattr(self.config, "verbose", False):
                print(f"[WikipediaRetriever] Search error: {e}")
            return []

        if not candidates:
            return []

        pid_list = list(candidates.keys())[:max(30, top_k * 10)]
        try:
            extracts = self.fetch_extracts(language, pid_list, chars=self.config.max_passage_length)
        except Exception as e:
            if getattr(self.config, "verbose", False):
                print(f"[WikipediaRetriever] Extract error: {e}")
            return []

        q_tokens = re.findall(r"\w+", question.lower())
        q_tokens = {t for t in q_tokens if len(t) >= 3}  # avoid noisy short tokens

        scored = []
        for pid in pid_list:
            title, snippet = candidates[pid]
            extract = extracts.get(pid, "") or ""
            title_l = title.lower()
            snippet_l = snippet.lower()
            extract_l = extract.lower()

            title_hits = sum(1 for t in q_tokens if t in title_l)
            snippet_hits = sum(1 for t in q_tokens if t in snippet_l)
            extract_hits = sum(1 for t in q_tokens if t in extract_l)

            score = 3 * title_hits + 2 * snippet_hits + 1 * extract_hits
            scored.append((score, pid, title, extract))

        scored.sort(reverse=True, key=lambda x: x[0])

        passages = []
        for score, pid, title, extract in scored[:top_k]:
            extract = (extract or "").strip()
            if extract:
                passages.append(f"[Source: {title}]\n{extract[:self.config.max_passage_length]}")

        return passages

    def retrieve(self, question: str, language: str, gold_context: Optional[str] = None) -> str:
        contexts = []

        if gold_context:
            contexts.append(f"[Gold Context]\n{gold_context}")

        wiki_passages = self.search_wikipedia(question, language, top_k=self.config.wikipedia_top_k)
        contexts.extend(wiki_passages)

        combined = "\n\n".join(contexts)
        return combined if combined else "No relevant context found."


## 5. Translation Module

In [6]:
class TranslationModule:
    """Handles cross-lingual translation between Indic languages and English"""

    def __init__(self, config: Config):
        self.config = config
        self.translator = Translator()

    def translate(self, text: str, source_lang: str, target_lang: str) -> str:
        """Translate text between languages"""
        if not text or source_lang == target_lang:
            return text

        try:
            src_code = self.config.language_codes.get(source_lang, source_lang)
            tgt_code = self.config.language_codes.get(target_lang, target_lang)

            # Handle long texts by splitting into chunks
            if len(text) > 5000:
                chunks = [text[i:i+4500] for i in range(0, len(text), 4500)]
                translated_chunks = []

                for chunk in chunks:
                    result = self.translator.translate(chunk, src=src_code, dest=tgt_code)
                    translated_chunks.append(result.text)
                    time.sleep(0.5)  # Rate limiting

                return " ".join(translated_chunks)
            else:
                result = self.translator.translate(text, src=src_code, dest=tgt_code)
                return result.text

        except Exception as e:
            if self.config.verbose:
                print(f"Translation error: {e}")
            return text  # Return original if translation fails

    def translate_to_english(self, text: str, source_lang: str) -> str:
        """Convenience method to translate to English"""
        return self.translate(text, source_lang, 'english')

    def translate_from_english(self, text: str, target_lang: str) -> str:
        """Convenience method to translate from English"""
        return self.translate(text, 'english', target_lang)

## 6. Gemma-based Self-RAG and Critique Module

In [7]:
class GemmaSelfRAG:
    """Gemma-2B-IT based regeneration and critique module"""

    def __init__(self, config: Config):
        self.config = config
        self.model = None
        self.tokenizer = None
        self._initialize_model()

    def _initialize_model(self):
        """Initialize Gemma model with optimized settings"""
        print(f"Loading {self.config.llm_model}...")

        # Configure quantization if needed
        bnb_config = None
        if self.config.load_in_4bit:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )
        elif self.config.load_in_8bit:
            bnb_config = BitsAndBytesConfig(
                load_in_8bit=True,
                bnb_8bit_compute_dtype=torch.float16
            )

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.llm_model,
            use_auth_token=True
        )

        # Set padding token if not present
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model with appropriate dtype and device mapping
        torch_dtype = torch.float16 if self.config.torch_dtype == "float16" else torch.float32

        self.model = AutoModelForCausalLM.from_pretrained(
            self.config.llm_model,
            torch_dtype=torch_dtype,
            device_map=self.config.device_map,
            quantization_config=bnb_config,
            use_auth_token=True,
            trust_remote_code=True
        )

        print(f"{Fore.GREEN}✓ Model loaded successfully!{Style.RESET_ALL}")

        # Print model info
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"Model parameters: {total_params / 1e9:.2f}B")


    def _format_prompt_gemma(self, instruction: str, context: str = "") -> str:
        """Format prompt for Gemma instruction-tuned model"""
        # Gemma-IT uses a specific format for instructions
        if context:
            prompt = f"<start_of_turn>user\n{context}\n\n{instruction}<end_of_turn>\n<start_of_turn>model\n"
        else:
            prompt = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n"
        return prompt

    def _generate_response(self, prompt: str) -> str:
        """Generate response using Gemma model"""

        try:
            # Tokenize input
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=2048
            )

            # Move to device
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=self.config.llm_max_new_tokens,
                    temperature=self.config.llm_temperature,
                    top_p=self.config.llm_top_p,
                    top_k=self.config.llm_top_k,
                    repetition_penalty=self.config.llm_repetition_penalty,
                    do_sample=True if self.config.llm_temperature > 0 else False,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )

            # Decode response
            response = self.tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            )

            return response.strip()

        except Exception as e:
            print(f"Generation error: {e}")

    def regenerate_answer(self, question: str, initial_answer: str, context: str, language: str) -> str:
        """Step 3: Self-RAG Regeneration in source language"""
        instruction = f"""You are a helpful assistant that answers questions based strictly on provided context.

Question: {question}

Initial Answer: {initial_answer}

Task: Re-answer the question using ONLY the information present in the context below.
If Gold Context is present, slightly prefer it over Wikipedia-based context when answering.
When using Wikipedia-based context, first compare it with the question and use it only if the context clearly supports it.
If the required information is missing or unclear in the context, explicitly state that you cannot answer due to lack of context.
Be factual and concise. Answer in {language}.

Your answer:"""

        formatted_prompt = self._format_prompt_gemma(instruction, context)
        regenerated = self._generate_response(formatted_prompt)

        if self.config.verbose:
            print(f"{Fore.CYAN}Regenerated answer in {language}{Style.RESET_ALL}")

        return regenerated

    def critique_answer(self, question_en: str, context_en: str, answer_en: str) -> CritiqueResult:
        """Step 5: Cross-lingual self-critique in English"""
        instruction = f"""You are a factual verification system. Analyze whether the given answer is supported by the provided context.

Question: {question_en}

Answer to verify: {answer_en}

Task: Provide your analysis in the following JSON format ONLY (no additional text):
{{
    "judgment": "supported" or "partially_supported" or "not_supported",
    "rationale": "Brief explanation citing specific evidence from context",
    "revised_answer": "A cautious, strictly context-based answer",
    "confidence": 0.0 to 1.0
}}

JSON Output:"""

        formatted_prompt = self._format_prompt_gemma(instruction, f"Context:\n{context_en}")
        response = self._generate_response(formatted_prompt)

        try:
            # Try to extract JSON from response
            # Sometimes models add extra text, so we try to find JSON
            json_start = response.find('{')
            json_end = response.rfind('}') + 1

            if json_start != -1 and json_end > json_start:
                json_str = response[json_start:json_end]
                critique_data = json.loads(json_str)
            else:
                # If no JSON found, try to parse entire response
                critique_data = json.loads(response)

            # Map string to enum
            judgment_map = {
                "supported": SupportJudgment.SUPPORTED,
                "partially_supported": SupportJudgment.PARTIALLY_SUPPORTED,
                "not_supported": SupportJudgment.NOT_SUPPORTED
            }

            return CritiqueResult(
                judgment=judgment_map.get(critique_data.get("judgment", "partially_supported"),
                                         SupportJudgment.PARTIALLY_SUPPORTED),
                rationale=critique_data.get("rationale", "Unable to determine support level"),
                revised_answer=critique_data.get("revised_answer", answer_en),
                confidence=float(critique_data.get("confidence", 0.5))
            )

        except Exception as e:
            print(f"Error parsing critique response: {e}")
            print(f"Raw response: {response[:200]}...") if self.config.verbose else None

            # Return default critique if parsing fails
            return CritiqueResult(
                judgment=SupportJudgment.PARTIALLY_SUPPORTED,
                rationale="Could not parse model response for verification",
                revised_answer=answer_en,
                confidence=0.5
            )

    def cleanup(self):
        """Clean up model from memory"""
        if self.model is not None:
            del self.model
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
        self.model = None
        self.tokenizer = None

## 8. Main Cross-Lingual Self-RAG Pipeline

In [8]:
class CrossLingualSelfRAG:
    """Main pipeline integrating all components with Gemma-2B-IT"""

    def __init__(self, config: Config = None):
        self.config = config or Config()

        # Initialize modules
        print("Initializing Cross-Lingual Self-RAG modules...")
        print("="*60)

        print("1. Initializing Wikipedia Retriever...")
        self.retriever = WikipediaRetriever(self.config)

        print("\n2. Initializing Translation Module...")
        self.translator = TranslationModule(self.config)

        print("\n3. Initializing Gemma-2B-IT Model...")
        self.llm = GemmaSelfRAG(self.config)

        print("="*60)
        print(f"{Fore.GREEN}✓ All modules initialized successfully!{Style.RESET_ALL}\n")

    def process(self,
                question_L: str,
                answer_L: str,
                language_L: str,
                context_L: Optional[str] = None) -> SelfRAGOutput:
        """
        Main processing pipeline

        Args:
            question_L: Question in Indic language
            answer_L: Initial answer in Indic language
            language_L: Source language (tamil/hindi/kannada/marathi)
            context_L: Optional gold context in Indic language

        Returns:

            with all results
        """

        print(f"\n{Fore.YELLOW}=== Processing Query in {language_L.upper()} ==={Style.RESET_ALL}")

        # Step 2: On-the-fly Wikipedia Retrieval
        #print(f"\n{Fore.BLUE}Step 2: Retrieving context...{Style.RESET_ALL}")
        retrieved_context = self.retriever.retrieve(question_L, language_L, context_L)

        # Step 3: Self-RAG Regeneration in source language
        #print(f"\n{Fore.BLUE}Step 3: Regenerating answer with context...{Style.RESET_ALL}")
        regenerated_answer_L = self.llm.regenerate_answer(
            question_L, answer_L, retrieved_context, language_L
        )

        # Step 4: Cross-lingual projection to English
        #print(f"\n{Fore.BLUE}Step 4: Translating to English...{Style.RESET_ALL}")
        question_EN = self.translator.translate_to_english(question_L, language_L)
        context_EN = self.translator.translate_to_english(retrieved_context, language_L)
        regenerated_answer_EN = self.translator.translate_to_english(regenerated_answer_L, language_L)

        # Step 5: Cross-lingual self-critique
        #print(f"\n{Fore.BLUE}Step 5: Running self-critique...{Style.RESET_ALL}")
        critique_result = self.llm.critique_answer(question_EN, context_EN, regenerated_answer_EN)

        # Step 6: Self-refinement via critique
        #print(f"\n{Fore.BLUE}Step 6: Refining answer based on critique...{Style.RESET_ALL}")
        if critique_result.judgment == SupportJudgment.SUPPORTED:
            final_answer_EN = regenerated_answer_EN
        else:
            final_answer_EN = critique_result.revised_answer

        # Step 7: Back-translation to source language
        #print(f"\n{Fore.BLUE}Step 7: Translating back to {language_L}...{Style.RESET_ALL}")
        final_answer_L = self.translator.translate_from_english(final_answer_EN, language_L)

        # Step 8: Prepare final output
        #print(f"\n{Fore.BLUE}Step 9: Preparing final output...{Style.RESET_ALL}")

        output = SelfRAGOutput(
            final_answer_indic=final_answer_L,
            judgment_label=critique_result.judgment.value,
            support_confidence=critique_result.confidence,
            english_rationale=critique_result.rationale,
            initial_answer=answer_L,
            retrieved_context=retrieved_context[:500] + "..." if len(retrieved_context) > 500 else retrieved_context,
            regenerated_answer=regenerated_answer_L,
            english_answer=final_answer_EN
        )

        print(f"\n{Fore.GREEN}✓ Processing complete!{Style.RESET_ALL}")

        return output

    def batch_process(self, data: List[Dict]) -> List[SelfRAGOutput]:
        """Process multiple examples"""
        results = []

        for idx, item in enumerate(tqdm(data, desc="Processing")):
            print(f"\n{Fore.MAGENTA}Processing item {idx + 1}/{len(data)}{Style.RESET_ALL}")

            result = self.process(
                question_L=item['question'],
                answer_L=item['answer'],
                language_L=item['language'],
                context_L=item.get('context', None)
            )

            results.append(result)

            # Save intermediate results if configured
            if self.config.save_intermediate:
                self._save_intermediate(results, f"intermediate_results_{idx+1}.json")

        return results

    def _save_intermediate(self, results: List[SelfRAGOutput], filename: str):
        """Save intermediate results to file"""
        data = [asdict(r) for r in results]
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def display_results(self, output: SelfRAGOutput):
        """Pretty print results"""
        print(f"\n{Fore.CYAN}{'='*60}{Style.RESET_ALL}")
        print(f"{Fore.YELLOW}FINAL RESULTS:{Style.RESET_ALL}")
        print(f"{Fore.CYAN}{'='*60}{Style.RESET_ALL}")

        print(f"\n{Fore.GREEN}Final Answer (Indic):{Style.RESET_ALL}")
        print(f"  {output.final_answer_indic}")

        print(f"\n{Fore.GREEN}Judgment:{Style.RESET_ALL} {output.judgment_label}")
        print(f"{Fore.GREEN}Confidence:{Style.RESET_ALL} {output.support_confidence:.2f}")

        if output.nli_score is not None:
            print(f"{Fore.GREEN}NLI Score:{Style.RESET_ALL} {output.nli_score:.2f}")

        print(f"\n{Fore.GREEN}Rationale:{Style.RESET_ALL}")
        print(f"  {output.english_rationale}")

        print(f"\n{Fore.GREEN}English Answer:{Style.RESET_ALL}")
        print(f"  {output.english_answer}")

        print(f"\n{Fore.CYAN}{'='*60}{Style.RESET_ALL}")

    def cleanup(self):
        """Clean up resources"""
        if hasattr(self, 'llm'):
            self.llm.cleanup()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

## 9. Test Data Generator

In [ ]:
def generate_test_data():
    """Generate non-trivial QA test data for 4 Indian languages (15 each)"""

    test_data = [

        # =========================
        # TAMIL (15)
        # =========================
        {"question": "திருக்குறளில் வள்ளுவர் அரசியலை எப்படி வரையறுக்கிறார்?", "answer": "திருக்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட்சியாக வரையறுக்கிறார்.", "language": "tamil", "context": "திருக்குறள் ஒரு நெறி நூல்."},
        {"question": "சிலப்பதிகாரத்தில் கண்ணகியின் பாத்திரத்தின் முக்கியத்துவம் என்ன?", "answer": "கண்ணகி நீதியின்象மாகவும் பெண்களின் வலிமையின்象மாகவும் விளங்குகிறாள்.", "language": "tamil", "context": None},
        {"question": "ஐம்பெரும் காப்பியங்களில் மணிமேகலையின் சமூக நோக்கம் என்ன?", "answer": "பசி ஒழிப்பு மற்றும் கருணை வளர்த்தல்.", "language": "tamil", "context": None},
        {"question": "தமிழகத்தில் சங்க இலக்கியம் உருவான காலப்பகுதியின் சிறப்பு என்ன?", "answer": "அது தமிழின் தொன்மையும் சமூக அமைப்பையும் பிரதிபலிக்கிறது.", "language": "tamil", "context": None},
        {"question": "பாரதி பெண்சாதிகுறித்து வைத்த பார்வை என்ன?", "answer": "பெண்கள் ஆண்களுக்கு சமமானவர்கள் என அவர் வலியுறுத்தினார்.", "language": "tamil", "context": None},
        {"question": "திராவிட இயக்கத்தின் சமூக நோக்கம் என்ன?", "answer": "சமத்துவம், சுயமரியாதை மற்றும் சமூக நீதியை நிலைநாட்டுதல்.", "language": "tamil", "context": None},
        {"question": "சிலப்பதிகாரத்தில் கோவலனின் வீழ்ச்சிக்கு காரணம் என்ன?", "answer": "தவறான முடிவுகளும் பேராசையும்.", "language": "tamil", "context": None},
        {"question": "தமிழக நீர்ப்பாசனத்தில் காவிரி ஆற்றின் பங்கு என்ன?", "answer": "முக்கியமான விவசாய ஆதாரமாக உள்ளது.", "language": "tamil", "context": None},
        {"question": "பக்தி இயக்கம் தமிழ்சமூகத்தில் ஏற்படுத்திய மாற்றம் என்ன?", "answer": "சாதி வேறுபாட்டைக் குறைத்து பக்தியை மையமாக்கியது.", "language": "tamil", "context": None},
        {"question": "தமிழ் திரைப்படங்களில் சமூக சிந்தனையின் வளர்ச்சி எப்படி?", "answer": "அவை சமூக பிரச்சினைகளை பிரதிபலிக்கத் தொடங்கின.", "language": "tamil", "context": None},
        {"question": "சோழர் கால நிர்வாகத்தின் சிறப்பம்சம் என்ன?", "answer": "உறுப்படையான உள்ளாட்சி அமைப்பு.", "language": "tamil", "context": None},
        {"question": "தமிழகத்தில் தொழில்துறை மயமாக்கலின் தாக்கம் என்ன?", "answer": "வேலைவாய்ப்பு அதிகரித்தது, நகரமயமாக்கல் ஏற்பட்டது.", "language": "tamil", "context": None},
        {"question": "திருக்குறளில் நட்பு பற்றிய கருத்து என்ன?", "answer": "நட்பு துன்பத்திலும் மாற்றமில்லாததாக இருக்க வேண்டும்.", "language": "tamil", "context": None},
        {"question": "தமிழ் மொழியின் செம்மொழி தகுதி ஏன் முக்கியம்?", "answer": "அதன் தொன்மையும் இலக்கிய வளமும் காரணம்.", "language": "tamil", "context": None},
        {"question": "தமிழக அரசியலில் மொழி அடையாளத்தின் பங்கு என்ன?", "answer": "அது மக்கள் அரசியலுடன் நெருக்கமாக இணைந்துள்ளது.", "language": "tamil", "context": None},

        # =========================
        # HINDI (15)
        # =========================
        {"question": "भारतीय संविधान के मौलिक अधिकारों का सामाजिक प्रभाव क्या है?", "answer": "इसने समानता और स्वतंत्रता को कानूनी संरक्षण दिया।", "language": "hindi", "context": None},
        {"question": "गांधीजी का सत्याग्रह आंदोलन क्यों प्रभावी था?", "answer": "क्योंकि यह अहिंसा और नैतिक बल पर आधारित था।", "language": "hindi", "context": None},
        {"question": "हिंदी साहित्य में प्रगतिवाद की मुख्य विशेषता क्या थी?", "answer": "सामाजिक यथार्थ और वर्ग संघर्ष।", "language": "hindi", "context": None},
        {"question": "भारतीय अर्थव्यवस्था में उदारीकरण का प्रभाव क्या रहा?", "answer": "निजीकरण बढ़ा और विदेशी निवेश आया।", "language": "hindi", "context": None},
        {"question": "जलवायु परिवर्तन का भारत पर दीर्घकालिक प्रभाव क्या हो सकता है?", "answer": "कृषि, जल संसाधन और तटीय क्षेत्रों पर गंभीर प्रभाव।", "language": "hindi", "context": None},
        {"question": "भारतीय संघीय प्रणाली की विशेषता क्या है?", "answer": "केंद्र और राज्यों के बीच शक्तियों का विभाजन।", "language": "hindi", "context": None},
        {"question": "नक्सलवाद की सामाजिक पृष्ठभूमि क्या है?", "answer": "गरीबी, असमानता और विकास की कमी।", "language": "hindi", "context": None},
        {"question": "भारतीय विदेश नीति में गुटनिरपेक्षता का क्या महत्व है?", "answer": "वैश्विक संतुलन बनाए रखना।", "language": "hindi", "context": None},
        {"question": "डिजिटल इंडिया कार्यक्रम का प्रमुख उद्देश्य क्या है?", "answer": "डिजिटल सेवाओं को आम नागरिक तक पहुँचाना।", "language": "hindi", "context": None},
        {"question": "भारतीय न्यायपालिका की स्वतंत्रता क्यों आवश्यक है?", "answer": "निष्पक्ष न्याय सुनिश्चित करने के लिए।", "language": "hindi", "context": None},
        {"question": "हिंदी सिनेमा में सामाजिक यथार्थवाद का विकास कब हुआ?", "answer": "1950–60 के दशकों में।", "language": "hindi", "context": None},
        {"question": "भारतीय कृषि संकट के प्रमुख कारण क्या हैं?", "answer": "जल संकट, कर्ज और मूल्य अस्थिरता।", "language": "hindi", "context": None},
        {"question": "राष्ट्रीय शिक्षा नीति का मुख्य उद्देश्य क्या है?", "answer": "गुणवत्तापूर्ण और समावेशी शिक्षा।", "language": "hindi", "context": None},
        {"question": "भारतीय समाज में जाति की भूमिका कैसे बदली है?", "answer": "यह धीरे-धीरे कमजोर हुई है पर पूर्णतः समाप्त नहीं।", "language": "hindi", "context": None},
        {"question": "महिला सशक्तिकरण भारत के विकास में क्यों आवश्यक है?", "answer": "क्योंकि यह आर्थिक और सामाजिक प्रगति को तेज करता है।", "language": "hindi", "context": None},

        # =========================
        # KANNADA (15)
        # =========================
        {"question": "ವಚನ ಸಾಹಿತ್ಯದ ಸಾಮಾಜಿಕ ಉದ್ದೇಶವೇನು?", "answer": "ಸಮಾನತೆ ಮತ್ತು ಜಾತಿ ವಿರೋಧ.", "language": "kannada", "context": None},
        {"question": "ಮೈಸೂರು ರಾಜವಂಶದ ಆಡಳಿತದ ವಿಶೇಷತೆ ಏನು?", "answer": "ಶಿಕ್ಷಣ ಮತ್ತು ಕೈಗಾರಿಕೆಯ ಬೆಂಬಲ.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದ ನೀರಾವರಿ ಸಮಸ್ಯೆಯ ಮೂಲ ಕಾರಣವೇನು?", "answer": "ಅನಿಯಮಿತ ಮಳೆಯೂ ನದಿಯ ವಿವಾದಗಳೂ.", "language": "kannada", "context": None},
        {"question": "ಕನ್ನಡ ನವೋದಯ ಸಾಹಿತ್ಯದ ಲಕ್ಷಣವೇನು?", "answer": "ಆಧುನಿಕ ಚಿಂತನೆ ಮತ್ತು ಸಮಾಜಮುಖಿ ಬರವಣಿಗೆ.", "language": "kannada", "context": None},
        {"question": "ಬೆಂಗಳೂರು ಐಟಿ ಹಬ್ ಆಗಲು ಕಾರಣವೇನು?", "answer": "ಶಿಕ್ಷಿತ ಮಾನವ ಸಂಪತ್ತು ಮತ್ತು ಹೂಡಿಕೆ.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದಲ್ಲಿ ಜಲಸಂರಕ್ಷಣೆಯ ಮಹತ್ವವೇನು?", "answer": "ಮುಂದಿನ ಪೀಳಿಗೆಗೆ ನೀರಿನ ಭದ್ರತೆ.", "language": "kannada", "context": None},
        {"question": "ರೈತ ಚಳವಳಿಗಳ ಸಾಮಾಜಿಕ ಪರಿಣಾಮವೇನು?", "answer": "ಸರಕಾರದ ನೀತಿಗಳ ಮೇಲೆ ಒತ್ತಡ.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದ ಭಾಷಾ ವೈವಿಧ್ಯತೆಯ ಮಹತ್ವವೇನು?", "answer": "ಸಾಂಸ್ಕೃತಿಕ ಸಹಬಾಳ್ವೆ.", "language": "kannada", "context": None},
        {"question": "ಮಾಧ್ಯಮಗಳ ಪ್ರಭಾವ ಸಮಾಜದ ಮೇಲೆ ಹೇಗೆ ಬೀರುತ್ತದೆ?", "answer": "ಅಭಿಪ್ರಾಯ ರೂಪಿಸುವಲ್ಲಿ ಪ್ರಮುಖ ಪಾತ್ರ.", "language": "kannada", "context": None},
        {"question": "ದಳಿತ ಸಾಹಿತ್ಯದ ಉದ್ದೇಶವೇನು?", "answer": "ಅನುಭವಗಳ ಮೂಲಕ ಶೋಷಣೆಯನ್ನು ಬಹಿರಂಗಪಡಿಸುವುದು.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದ ಪ್ರವಾಸೋದ್ಯಮದ ಆರ್ಥಿಕ ಲಾಭವೇನು?", "answer": "ಉದ್ಯೋಗ ಮತ್ತು ಆದಾಯ ಹೆಚ್ಚಳ.", "language": "kannada", "context": None},
        {"question": "ಪಂಚಾಯತ್ ರಾಜ್ ವ್ಯವಸ್ಥೆಯ ಉದ್ದೇಶವೇನು?", "answer": "ಗ್ರಾಮೀಣ ಆಡಳಿತಕ್ಕೆ ಶಕ್ತಿ ವಹಿಸುವುದು.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದ ಅರಣ್ಯ ಸಂಪತ್ತು ರಕ್ಷಣೆಯ ಅಗತ್ಯವೇನು?", "answer": "ಪರಿಸರ ಸಮತೋಲನಕ್ಕಾಗಿ.", "language": "kannada", "context": None},
        {"question": "ಶಿಕ್ಷಣದಲ್ಲಿ ತಂತ್ರಜ್ಞಾನ ಬಳಕೆಯ ಲಾಭವೇನು?", "answer": "ವಿಸ್ತೃತ ಪ್ರವೇಶ ಮತ್ತು ಸುಲಭ ಅಧ್ಯಯನ.", "language": "kannada", "context": None},
        {"question": "ಕರ್ನಾಟಕದ ಭಾರತದ ಒಕ್ಕೂಟದಲ್ಲಿ ಪಾತ್ರವೇನು?", "answer": "ಉದ್ಯಮ, ತಂತ್ರಜ್ಞಾನ ಮತ್ತು ಸಾಂಸ್ಕೃತಿಕ ಕೊಡುಗೆ.", "language": "kannada", "context": None},

        # =========================
        # MARATHI (15)
        # =========================
        {"question": "शिवाजी महाराजांच्या राज्यकारभाराची वैशिष्ट्ये कोणती?", "answer": "लोककल्याणकारी प्रशासन आणि सैन्यशिस्त.", "language": "marathi", "context": None},
        {"question": "महाराष्ट्राच्या औद्योगिकीकरणाचा परिणाम काय झाला?", "answer": "रोजगारवाढ आणि शहरविकास.", "language": "marathi", "context": None},
        {"question": "साहित्य चळवळींनी समाजावर कसा प्रभाव टाकला?", "answer": "सामाजिक प्रबोधन आणि परिवर्तन.", "language": "marathi", "context": None},
        {"question": "सहकार चळवळीचे आर्थिक महत्त्व काय आहे?", "answer": "ग्रामीण अर्थव्यवस्थेला बळकटी.", "language": "marathi", "context": None},
        {"question": "पाणी टंचाईची प्रमुख कारणे कोणती?", "answer": "अनियमित पर्जन्य आणि अपव्यवहार.", "language": "marathi", "context": None},
        {"question": "शिक्षणात डिजिटल माध्यमाचे फायदे कोणते?", "answer": "सोपी उपलब्धता आणि व्यापक पोहोच.", "language": "marathi", "context": None},
        {"question": "मराठी रंगभूमीचे सामाजिक योगदान काय आहे?", "answer": "सामाजिक प्रश्नांची जाणीव.", "language": "marathi", "context": None},
        {"question": "शहरीकरणाचे पर्यावरणावर परिणाम काय आहेत?", "answer": "प्रदूषण आणि हरितक्षेत्र घट.", "language": "marathi", "context": None},
        {"question": "स्वातंत्र्य चळवळीत महाराष्ट्राचे योगदान काय होते?", "answer": "क्रांतिकारी आणि समाजसुधारक नेतृत्व.", "language": "marathi", "context": None},
        {"question": "महिला स्वयं-सहायता गटांचे महत्त्व काय आहे?", "answer": "आर्थिक स्वावलंबन.", "language": "marathi", "context": None},
        {"question": "भारतीय लोकशाहीत माध्यमांची भूमिका काय आहे?", "answer": "सत्तेवर नियंत्रण आणि जनमत निर्माण.", "language": "marathi", "context": None},
        {"question": "शेतीमध्ये तंत्रज्ञानाचा उपयोग कसा वाढतो आहे?", "answer": "उत्पादनक्षमता आणि कार्यक्षमता.", "language": "marathi", "context": None},
        {"question": "मराठी भाषेच्या संवर्धनाचे महत्त्व काय आहे?", "answer": "सांस्कृतिक ओळख टिकवण्यासाठी.", "language": "marathi", "context": None},
        {"question": "ग्रामीण बेरोजगारी वाढण्याची कारणे कोणती?", "answer": "शेतीवरील अवलंबित्व आणि संधींचा अभाव.", "language": "marathi", "context": None},
        {"question": "पर्यावरण संरक्षणासाठी जनजागृती का आवश्यक आहे?", "answer": "शाश्वत विकासासाठी.", "language": "marathi", "context": None},
    ]

    return test_data


print("Test data generator created!")
print(f"Total samples generated: {len(generate_test_data())}")


Test data generator created!
Total samples generated: 60


## 10. Running the Pipeline

In [9]:
# Configure the pipeline
config = Config(
    verbose=False,
    save_intermediate=True,
    wikipedia_top_k=3,
    llm_temperature=0.1,
    llm_max_new_tokens=256,
)

# Create pipeline instance
print("Initializing pipeline with Gemma-2B-IT...\n")
pipeline = CrossLingualSelfRAG(config=config)

print("\n" + "="*60)
print("Pipeline ready for processing!")
print("="*60)

Initializing pipeline with Gemma-2B-IT...

Initializing Cross-Lingual Self-RAG modules...
1. Initializing Wikipedia Retriever...

2. Initializing Translation Module...

3. Initializing Gemma-2B-IT Model...
Loading google/gemma-2b-it...


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

✓ Model loaded successfully!
Model parameters: 2.51B
✓ All modules initialized successfully!


Pipeline ready for processing!


##Test Data

In [ ]:
# Test with a single example
test_data = generate_test_data()
sample_input = test_data[0]  # Tamil example

print(f"Testing with {sample_input['language']} example:")
print(f"Question: {sample_input['question']}")
print(f"Initial Answer: {sample_input['answer']}")
print(f"Has Context: {'Yes' if sample_input['context'] else 'No'}")

# Process single example
result = pipeline.process(
    question_L=sample_input['question'],
    answer_L=sample_input['answer'],
    language_L=sample_input['language'],
    context_L=sample_input.get('context')
)

# Display results
pipeline.display_results(result)

Testing with tamil example:
Question: திருக்குறளில் வள்ளுவர் அரசியலை எப்படி வரையறுக்கிறார்?
Initial Answer: திருக்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட்சியாக வரையறுக்கிறார்.
Has Context: Yes

=== Processing Query in TAMIL ===

✓ Processing complete!

FINAL RESULTS:

Final Answer (Indic):
  தார்க்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட்சி என வரையறுக்கிறார்.

Judgment: supported
Confidence: 1.00

Rationale:
  The passage explicitly states that Valluvar defines politics as rule based on virtue.

English Answer:
  Valluvar in Tarkurkural defines politics as rule based on virtue.



In [ ]:
# Process all test examples
test_data = generate_test_data()
print(f"Processing {len(test_data)} test examples...\n")

results = pipeline.batch_process(test_data)

# Display summary
print(f"\n{Fore.YELLOW}SUMMARY OF RESULTS:{Style.RESET_ALL}")
print(f"{Fore.CYAN}{'='*60}{Style.RESET_ALL}")

for idx, (input_data, result) in enumerate(zip(test_data, results)):
    print(f"\n{idx+1}. {input_data['language'].upper()}:")
    print(f"   Question: {input_data['question'][:50]}..." if len(input_data['question']) > 50 else f"   Question: {input_data['question']}")
    print(f"   Judgment: {result.judgment_label}")
    print(f"   Confidence: {result.support_confidence:.2f}")
    print(f"   Final Answer: {result.final_answer_indic[:50]}..." if len(result.final_answer_indic) > 50 else f"   Final Answer: {result.final_answer_indic}")

Processing 60 test examples...



Processing:   0%|          | 0/60 [00:00<?, ?it/s]


Processing item 1/60

=== Processing Query in TAMIL ===

✓ Processing complete!


Processing:   2%|▏         | 1/60 [00:06<06:00,  6.11s/it]


Processing item 2/60

=== Processing Query in TAMIL ===

✓ Processing complete!


...


Processing item 59/60

=== Processing Query in MARATHI ===

✓ Processing complete!


Processing:  98%|█████████▊| 59/60 [09:49<00:11, 11.37s/it]


Processing item 60/60

=== Processing Query in MARATHI ===

✓ Processing complete!


Processing: 100%|██████████| 60/60 [10:05<00:00, 10.09s/it]



SUMMARY OF RESULTS:

1. TAMIL:
   Question: திருக்குறளில் வள்ளுவர் அரசியலை எப்படி வரையறுக்கிறா...
   Judgment: supported
   Confidence: 1.00
   Final Answer: தார்க்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட்சி ...

2. TAMIL:
   Question: சிலப்பதிகாரத்தில் கண்ணகியின் பாத்திரத்தின் முக்கிய...
   Judgment: partially_supported
   Confidence: 0.50
   Final Answer: எச்சரிக்கையான, கண்டிப்பான சூழல் சார்ந்த பதில்

3. TAMIL:
   Question: ஐம்பெரும் காப்பியங்களில் மணிமேகலையின் சமூக நோக்கம்...
   Judgment: partially_supported
   Confidence: 0.70
   Final Answer: ஒரு எச்சரிக்கையான, கண்டிப்பான சூழல் அடிப்படையிலான ...

4. TAMIL:
   Question: தமிழகத்தில் சங்க இலக்கியம் உருவான காலப்பகுதியின் ச...
   Judgment: partially_supported
   Confidence: 0.50
   Final Answer: இக்காலப் பகுதியின் சமூக, பண்பாட்டு அம்சங்களைச் சுர...

5. TAMIL:
   Question: பாரதி பெண்சாதிகுறித்து வைத்த பார்வை என்ன?
   Judgment: partially_supported
   Confidence: 0.50
   Final Answer: பெண்களின் சாதி பற்றிய எதிர்மறையான கருத்தைச் சூழல் ...

## 11. Export Results

In [ ]:
# Save results to JSON
output_filename = "gemma_cross_lingual_self_rag_results.json"

# Convert results to dictionaries
export_data = []
for input_data, result in zip(test_data, results):
    export_item = {
        "input": input_data,
        "output": asdict(result)
    }
    export_data.append(export_item)

# Save to file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"Results saved to {output_filename}")

# Also save as CSV for easy viewing
df_results = pd.DataFrame([
    {
        'language': input_data['language'],
        'question': input_data['question'],
        'initial_answer': input_data['answer'],
        'final_answer': result.final_answer_indic,
        'judgment': result.judgment_label,
        'confidence': result.support_confidence,
        'has_context': 'Yes' if input_data.get('context') else 'No'
    }
    for input_data, result in zip(test_data, results)
])

csv_filename = "gemma_cross_lingual_self_rag_results.csv"
df_results.to_csv(csv_filename, index=False, encoding='utf-8')
print(f"Results also saved to {csv_filename}")

# Display DataFrame
print("\nResults Summary:")
df_results

Results saved to gemma_cross_lingual_self_rag_results.json
Results also saved to gemma_cross_lingual_self_rag_results.csv

Results Summary:


,language,question,initial_answer,final_answer,judgment,confidence,has_context
0,tamil,திருக்குறளில் வள்ளுவர் அரசியலை எப்படி வரையறுக்...,திருக்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட...,தார்க்குறளில் வள்ளுவர் அரசியலை அறம் சார்ந்த ஆட...,supported,1.0,Yes
1,tamil,சிலப்பதிகாரத்தில் கண்ணகியின் பாத்திரத்தின் முக...,கண்ணகி நீதியின்象மாகவும் பெண்களின் வலிமையின்象மா...,"எச்சரிக்கையான, கண்டிப்பான சூழல் சார்ந்த பதில்",partially_supported,0.5,No
2,tamil,ஐம்பெரும் காப்பியங்களில் மணிமேகலையின் சமூக நோக...,பசி ஒழிப்பு மற்றும் கருணை வளர்த்தல்.,"ஒரு எச்சரிக்கையான, கண்டிப்பான சூழல் அடிப்படையி...",partially_supported,0.7,No
3,tamil,தமிழகத்தில் சங்க இலக்கியம் உருவான காலப்பகுதியி...,அது தமிழின் தொன்மையும் சமூக அமைப்பையும் பிரதிப...,"இக்காலப் பகுதியின் சமூக, பண்பாட்டு அம்சங்களைச்...",partially_supported,0.5,No
4,tamil,பாரதி பெண்சாதிகுறித்து வைத்த பார்வை என்ன?,பெண்கள் ஆண்களுக்கு சமமானவர்கள் என அவர் வலியுறு...,பெண்களின் சாதி பற்றிய எதிர்மறையான கருத்தைச் சூ...,partially_supported,0.5,No
5,tamil,திராவிட இயக்கத்தின் சமூக நோக்கம் என்ன?,"சமத்துவம், சுயமரியாதை மற்றும் சமூக நீதியை நிலை...",சுயமரியாதை மற்றும் சமூக நீதியை வளர்ப்பதே திராவ...,supported,1.0,No
6,tamil,சிலப்பதிகாரத்தில் கோவலனின் வீழ்ச்சிக்கு காரணம்...,தவறான முடிவுகளும் பேராசையும்.,"எச்சரிக்கையான, கண்டிப்பான சூழல் சார்ந்த பதில்",partially_supported,0.5,No
7,tamil,தமிழக நீர்ப்பாசனத்தில் காவிரி ஆற்றின் பங்கு என்ன?,முக்கியமான விவசாய ஆதாரமாக உள்ளது.,தமிழ்நாட்டின் விவசாய பாசனத்திற்கு காவிரி ஆறு ம...,supported,1.0,No
8,tamil,பக்தி இயக்கம் தமிழ்சமூகத்தில் ஏற்படுத்திய மாற்...,சாதி வேறுபாட்டைக் குறைத்து பக்தியை மையமாக்கியது.,தமிழ் சமூகத்தில் தமிழ் பக்தி இயக்கம் ஏற்படுத்த...,supported,1.0,No
9,tamil,தமிழ் திரைப்படங்களில் சமூக சிந்தனையின் வளர்ச்ச...,அவை சமூக பிரச்சினைகளை பிரதிபலிக்கத் தொடங்கின.,தமிழ் திரைப்படங்களில் சமூக ஒற்றுமையின் வளர்ச்ச...,partially_supported,0.5,No


##Plug and Play

In [29]:
import os
import pandas as pd
from tqdm import tqdm

def run_selfrag_on_csv(input_csv_path, pipeline):
    """
    input_csv_path: path to a CSV under Results/Vanilla/...
    pipeline: initialized Self-RAG pipeline
    """

    # ---------- load ----------
    df = pd.read_csv(input_csv_path)

    required_cols = {"Question", "model_response"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # ---------- run self-rag ----------
    final_answers = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        question = row["Question"]
        initial_answer = row["model_response"]

        # Adjust this call ONLY if your pipeline signature differs
        result = pipeline.process(
            question_L=question,
            answer_L=initial_answer,
            language_L="tamil",
            context_L=None
        )

        # If pipeline returns dict/string, normalize
        if isinstance(result, dict):
            final_answer = result.get("final_answer", result.get("answer", ""))
        else:
            final_answer = result

        final_answers.append(final_answer)

    df["final_answer"] = final_answers

    # ---------- output path ----------
    output_csv_path = input_csv_path.replace(
        os.sep + "Vanilla" + os.sep,
        os.sep + "selfrag" + os.sep
    )

    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

    # ---------- save ----------
    df.to_csv(output_csv_path, index=False)

    return output_csv_path


In [ ]:
input_csv = "/content/drive/MyDrive/Project/Dataset/IndicQuest/Results/Vanilla/aya/aya_results_Tamil.csv"

output_csv = run_selfrag_on_csv(input_csv, pipeline)
print("Saved to:", output_csv)


In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive
